# Optimization

There are many engineering problems that can be expressed as an optimization of parameters to minimize/maximize some function. 

**Cute trick: You can turn any maximization problem into a minimization one by multiplying by -1... **

For example, what angle should you put the wind turbine blades if the wind is blowing at 30 mph if you want to maximize energy output? 

Optimization assumes you have some function/simulation you can run that will output the amount of energy produced given the angle and the wind speed. Then it's a "search" for the angle that maximizes the energy produced. The simplest form of search is gradient descent.

In this tutorial we'll focus on the mechanics of setting up the search using a quadratic function (find **x** given **f(x)**) and a 2D parameter search (find **x,y** given **f(x,y)**). The **f** is just a generic function that you could solve for analytically; but pretend it's some complicated bit of code that simulates some system.

In [ ]:
# Our usual imports
import numpy as np
import matplotlib.pyplot as plt

# New scipy import that finds the minima of a function
from scipy.optimize import fmin
# A helper function that "binds" variables
from functools import partial


In [ ]:
# A generic function that we want to find the minimum of
def my_func(x):
    """A quadratic function
    @param x the input x value
    @returns f(x)"""
    return (x-2)**2 - 3

## Use fmin to find the minimum of my_func

The _ says ignore that returned value

See https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin.html for what the remaining values are

Note that **x_at_min** is a list - in this case, a list of dimension 1

The **full_output=True** prints out the optimization result to the console

TODO: look in the list of parameters for fmin to find how to control the desired accuracy of the answer and how many iterations to try

In [ ]:
# The fmin call
x_at_min, f_at_x_min, _, _, _ = fmin(my_func, x0=0.2, maxfun=100, full_output=True)

# Notice the [0] to get the first element out of the x list
print(f"Minimum of f is {f_at_x_min}, happens at x={x_at_min[0]}")
print(f"Checking fmin result {f_at_x_min} against func eval {my_func(x_at_min[0])}")

In [ ]:
# Plot the function and the result
fig, axs = plt.subplots(1, 1, figsize=(4, 4))
ts = np.linspace(-1.0, 3.0)
axs.plot(ts, my_func(ts), '-k', label='Quadratic')
axs.plot(x_at_min, f_at_x_min, 'Xr')
axs.set_xlabel('x')
axs.set_ylabel('y')
axs.axis('equal')
axs.set_title("Minimum of quadratic")

## Example with parameters

Now going to do the same thing, but with the equation having parameters that get set, and using fmin's ability to pass parameters in to make this a general search over any parabola

In [ ]:
# A generic function that we want to find the minimum of
def my_func_with_params(x, a, b, c):
    """A quadratic function a x^2 + bx + c
    @param x the input x value
    @param a a x^2
    @param b bx
    @param c c
    @returns a x^2 + bx + c"""
    return a * x**2 + b * x + c

## Passing parameters to your function through fmin

Because the function **my_func_with_params** definition has all these three extra parameters that control the shape of the quadratic (**a, b, c**), we have do what's called "binding" the variables to values, in this case **a = 3, b = 2, c = -16**

We're going to do this in two ways; one is using Python's **lambda** functionality to create a new (unnamed) function that only takes in one parameter. The second is passing the paramters through to **my_func_with_params** using **fmin's** **args** option.

## With lambda...
**fmin** takes a function that takes in one argument (a list of dimension d) and outputs one number. Our **my_func_with_params** function currently takes in **x, a, b, c**, not **x**.

**Lambda** functions fix that: In this case, **lambda** says make a new (temporary, unnamed function) that takes in one parameter (**array**) and calls **my_func_with_params** with **x**

Notice this time we saved the returned tuple to result, rather than unpacking it

In [ ]:
# the quadratic we want to search for the minimum of
a = 3 
b = 2
c = -16
result = fmin(lambda x: my_func_with_params(x, a=a, b=b, c=c), x0=0, maxfun=200, full_output=True)
print(result)
print(f"Function minimum is {result[1]}, found at x {result[0]}")

In [ ]:
# Breaking this down a little - you use lambda functions so you don't *have* to name the function, 
# but we can go ahead and name it

my_func_no_params = lambda x: my_func_with_params(x, a=a, b=b, c=c)

# These are exactly the same
res_1 = my_func_with_params(x=0.0, a=a, b=b, c=c)
res_2 = my_func_no_params(x=0.0)

print(f"Res_1 {res_1} and res_2 {res_2} are the same")

## Using fmin args
Using **fmin**'s args function to pass parameters directly to the function.

The fancy term(s) for this is packing and unpacking arguments, but you can think of it as just passing the variables through. A lot of scipy/numpy functions that take functions have this ability.

In [ ]:
# args must be in the same order as the additional parameters in my_func_with_params
#   fmin just "unpacks" the tuple into parameters 2,3, and 4 in the function
# TODO: Change args so it only has 2 (or 4) things in it (eg, (a,b)). What error message do you get? Why?
result = fmin(my_func_with_params, x0=0.0, args=(a, b, c), full_output=True)
print(f"Function minimum is {result[1]}, found at x {result[0]}")

# Error with (a,b):
#   TypeError: my_func_with_params() missing 1 required positional argument: 'c'
# Error with (a,b,c,4):
#   TypeError: my_func_with_params() takes 4 positional arguments but 5 were given